In [1]:
import sys
sys.path.append('../scared_lbc')

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import cupy as _cu
from tqdm.auto import tqdm
import numpy as np
import math

In [4]:
import scared_cu

In [5]:
47//8

5

In [6]:
reduction = scared_cu.lbc.modop.Reduction_0Q(257, 'int16')

model = scared_cu.models.SignedHammingWeight(expected_dtype='int16')
# model = scared_cu.models.ShiftRight(4)
# model = ShiftRightAndMask(6, 6)
# model = BinHammingWeight(expected_dtype='int32')
# model = scared_cu.models.Mask(6)
# model = scared_cu.Value()

partitions = _cu.arange(9)


N = 15000
sigma = 1
conv = 3000

n = 32 #don't touch
n_step = 32

neg_trick = False

prng_off = False

incomplete = 0

In [7]:
def mmia_wrapper(ths, reduction, model, partitions,step=1000, conv_step=1000, neg_trick=False, subkeys=None, early_stop=True, pp=False, accept_shift=False, sa=2):

    if incomplete == 0:
        selection_function = scared_cu.lbc.selection_functions.BaseMul(reduction=reduction, words=subkeys)
    else:
        selection_function = scared_cu.lbc.selection_functions.BaseMulIncomplete(reduction=reduction, words=subkeys)


    if reduction.o_dtype == 'int8':
        bin_edges = [-0.5,0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5]
    elif reduction.o_dtype == 'int16':
        bin_edges = [-0.5,0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5,12.5,13.5,14.5,15.5,16.5]
    elif reduction.o_dtype == 'int32':
        bin_edges = [-0.5,0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5,12.5,13.5,14.5,15.5,16.5,17.5,18.5,19.5,20.5,21.5,22.5,23.5,24.5,25.5,26.5,27.5,28.5,29.5,30.5,31.5,32.5]
    bin_edges = [-0.5,0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5,12.5,13.5,14.5,15.5,16.5,17.5,18.5,19.5,20.5,21.5,22.5,23.5,24.5,25.5,26.5,27.5,28.5,29.5,30.5,31.5,32.5]
    bin_edges = [-0.5,0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5]

    mmia = scared_cu.MMIAAttack(selection_function=selection_function,
                            model=model,
                            bin_edges=bin_edges,
                            partitions=partitions,
                            same_bins=True,
                            place_outliers=False,
                            offset=n, 
                            discriminant=scared_cu.maxabs,
                            convergence_step=conv_step)


    for i in tqdm(range(0, len(ths), step)):
        container = scared_cu.Container(ths[i:i + step])
        mmia.run(container)

        if early_stop:
            succs = scared_cu._utils.succ_ratio(ths.s[0, subkeys], reduction.q, conv, mmia, accept_shift=accept_shift, sa=sa, incomplete=incomplete!=0)
            if succs[-1] == 1.0:
                break


    return mmia

In [8]:
ths = scared_cu.lbc.simulated_traces.collect_traces_basemult(N=N, n=n, sigma=sigma, reduction=reduction, incomplete=incomplete,
                              const_seed=True, two_step=False, seed=0, prng_off=prng_off, leak_reduction=True)

print(ths)

  0%|          | 0/15000 [00:00<?, ?it/s]

Trace Header Set:
Name.............: RAM Format THS
Reader...........: RAM reader with 15000 traces. Samples shape: (15000, 64) and metadatas: ['c', 's', 's0', 's1']
c................: int16
s................: int16
s0...............: int16
s1...............: int16



In [9]:
succs = None
for n_i in tqdm(range(0, n, n_step)):
    subkeys = _cu.arange(n_i, n_i + n_step)
    assert ((ths.s % reduction.q) == ((ths.s0 + ths.s1) % reduction.q)).all()
    mmia = mmia_wrapper(ths[:N], reduction, model, partitions, step=conv, conv_step=conv, neg_trick=neg_trick, subkeys=subkeys, early_stop=False, pp=not prng_off)
    succs_i_ = scared_cu._utils.succ_ratio(ths.s[0, subkeys], reduction.q, conv, mmia, incomplete=incomplete!=0)
    succs_i = _cu.zeros(N//conv)
    for i in range(len(succs_i_)):
        succs_i[i] = succs_i_[i]
    for i in range(len(succs_i_), len(succs_i)):
        succs_i[i] = succs_i_[-1]
    if succs is None:
        succs = _cu.array(succs_i)
    else:
        succs += _cu.array(succs_i)
    
    for x,y in zip(range(conv,N+1,conv),succs):
        print(f'({math.log(x)/math.log(10)},{y})',end='')
    # if n_i != (n - n_step):
    #     del kw
    print()
    for i in range(n_i, n_i + n_step):
        print(f'{ths.s[0, i]}-{mmia.scores[:,i-n_i].argsort()[-1]}', end='\t')
    print()


succs /= (n//n_step)
# assert (mmia.data == (-1)).sum() == 0


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

(3.4771212547196617,0.03125)(3.778151250383643,0.1875)(3.9542425094393248,0.5625)(4.079181246047624,0.875)(4.176091259055681,0.875)
183-183	184-184	42-42	254-254	47-47	60-60	193-193	203-203	242-227	6-3	48-48	221-239	234-234	242-242	170-170	97-97	220-220	142-142	125-125	33-33	67-67	67-67	201-201	157-157	86-86	133-133	171-171	42-42	240-240	7-7	235-246	105-105	


In [10]:
ths.samples.min()

array(-3.972682)

In [11]:
for x,y in zip(range(conv,N+1,conv),succs):
    print(f'({math.log(x)/math.log(10)},{y})',end='')

(3.4771212547196617,0.03125)(3.778151250383643,0.1875)(3.9542425094393248,0.5625)(4.079181246047624,0.875)(4.176091259055681,0.875)

In [12]:
reduction = scared_cu.lbc.modop.Reduction_0Q(4, 'int16')
guesses_coeff = reduction.reduce(_cu.arange(0, reduction.q, dtype=reduction.o_dtype))
high = _cu.repeat(guesses_coeff, reduction.q)
low = _cu.tile(guesses_coeff, reduction.q)
guesses = _cu.concatenate((low[:, _cu.newaxis], high[:, _cu.newaxis]), axis=1)
print(guesses)
print()
c_ = _cu.array([[[0, 1], [2, 3]],[[0, 2], [2, 2]],[[2, 0], [3, 2]]], dtype=reduction.o_dtype)
res = reduction.reduce((c_[:, _cu.newaxis, :, :] * guesses[_cu.newaxis, :, _cu.newaxis, :]).sum(axis=3))
print(res)

[[0 0]
 [1 0]
 [2 0]
 [3 0]
 [0 1]
 [1 1]
 [2 1]
 [3 1]
 [0 2]
 [1 2]
 [2 2]
 [3 2]
 [0 3]
 [1 3]
 [2 3]
 [3 3]]

[[[0 0]
  [0 2]
  [0 0]
  [0 2]
  [1 3]
  [1 1]
  [1 3]
  [1 1]
  [2 2]
  [2 0]
  [2 2]
  [2 0]
  [3 1]
  [3 3]
  [3 1]
  [3 3]]

 [[0 0]
  [0 2]
  [0 0]
  [0 2]
  [2 2]
  [2 0]
  [2 2]
  [2 0]
  [0 0]
  [0 2]
  [0 0]
  [0 2]
  [2 2]
  [2 0]
  [2 2]
  [2 0]]

 [[0 0]
  [2 3]
  [0 2]
  [2 1]
  [0 2]
  [2 1]
  [0 0]
  [2 3]
  [0 0]
  [2 3]
  [0 2]
  [2 1]
  [0 2]
  [2 1]
  [0 0]
  [2 3]]]


In [13]:
mmia.scores.shape

(257, 32)